<a href="https://colab.research.google.com/github/zoltanzack/CS3TM20/blob/master/CW_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

# **Steps outline**
1. Download your data set by inputting your student number.
2. Process your text data, extract features, convert them into vectors
3. Modeling, train models on the data set (select model, tune different parameters)
4. Process your text data, extract features, convert them into vectors
5. Analysis and discussions

# Step 1: Load Dataset


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',  categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',  categories=categories, shuffle=True, random_state=42)

**This is how to identify which data set to use (Please copy  the following information in report front   page).**

In [ ]:
index=input('type your student number?')

type your student number?12345678


In [ ]:
x=divmod(int(index),4)
yourdata1=x[1]
y=divmod(int(index),3)
yourdata2=y[1]
print('This is your data set index ----> (', x[1], y[1], ')' )

This is your data set index ----> ( 2 0 )


In [ ]:
data1= twenty_train.target_names[x[1]]
data2= twenty_train.target_names[y[1]]
categories1=[data1,data2]
print(categories1)

['sci.med', 'alt.atheism']


**Your front page data information Ends here**

# Step 2 Process your text data, extract features

# 2.1 An example of preprocessing

**An example is provided. You have modify to apply to data set based on your student number.**

In [ ]:
# write your own NLP precessing examples with  preprocessing techniques.

dataset=twenty_train.data[6]
print(dataset)
# please use own data set by replacing 6 to one of yours.

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# tokenize: search: nltk tokenize 
example = "This is an example sentence."

from nltk.tokenize import word_tokenize
example_tokenize= word_tokenize(example) #twenty_train.data[1]
print("-------------------------tokenize:")
print(example_tokenize)


# stemmer: search: nltk stemmer  
stemmer = nltk.stem.PorterStemmer()
example_stem = stemmer.stem(example)
print("-------------------------stem:")
print(example_stem)

# tf-idf: search: scikit learn tf-idf https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
# search scikit learn CountVectorizer example: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 

# pos_taging: search: nltk pos tagging example
example_posTag=nltk.pos_tag(example_tokenize)
print("-------------------------pos_taging:")
print(example_posTag)

# consituency parsing, chunking
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(example_posTag)
print(result)




From: jodfishe@silver.ucs.indiana.edu (joseph dale fisher)
Subject: Re: anger
Organization: Indiana University
Lines: 34

In article <Apr.17.01.10.44.1993.2232@geneva.rutgers.edu> news@cbnewsk.att.com writes:
>>Paul Conditt writes:
[insert deletion of Paul's and Aaron's discourse on anger, ref Galatians
5:19-20]
>
>I don't know why it is so obvious.  We are not speaking of acts of the 
>flesh.  We are just speaking of emotions.  Emotions are not of themselves
>moral or immoral, good or bad.  Emotions just are.  The first step is
>not to label his emotion as good or bad or to numb ourselves so that
>we hide our true feelings, it is to accept ourselves as we are, as God
>accepts us.  

Oh, but they definitely can be.  Please look at Colossians 3:5-10 and
Ephesians 4:25-27.  Emotions can be controlled and God puts very strong
emphasis on self-control, otherwise, why would he have Paul write to
Timothy so much about making sure to teach self-control? 

[insert deletion of remainder of para

#2.2 NLP Preprocesssing

**Some preprocessing are provided for convenience. Please include why NLP preprocessing is in your report. Explain what techniques have been applied in your coursework.**

In [ ]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from tqdm import tqdm
from nltk.corpus import stopwords
stopwordEn = stopwords.words('english')
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

def lemmaWord(word):
    lemma = wordnet.morphy(word)
    if lemma is not None:
        return lemma
    else:
        return word

def stemWord(word):
    stem = stemmer.stem(word)
    if stem is not None:
        return stem
    else:
        return word

def processText(text,lemma=False, gram=1, rmStop=True): # default remove stop words
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|@\w+|#', '', text, flags=re.MULTILINE) #delete URL, #hashtag# , and @xxx
    tokens = word_tokenize(text)
    whitelist = ["n't", "not", "no"]
    new_tokens = []
    stoplist = stopwordEn if rmStop else []
    for i in tokens:
      i = i.lower()
      if i.isalpha() and (i not in stoplist or i in whitelist):  #i not in ['.',',',';']  and (...)
        if lemma: i = lemmaWord(i)
        new_tokens.append(i)
    del tokens
    # tokens = [lemmaWord(i.lower()) if lemma else i.lower() for i in tokens if (i.lower() not in stoplist or i.lower() in whitelist) and i.isalpha()]
    if gram<=1:
        return new_tokens
    else:
        return [' '.join(i) for i in nltk.ngrams(new_tokens, gram)]

       

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def getTags(text):
  token = word_tokenize(text)
  token = [l.lower() for l in token]
  train_tags = nltk.pos_tag(token)
  return [i[1] for i in train_tags]


In [ ]:
print(processText(dataset))

['joseph', 'dale', 'fisher', 'subject', 'anger', 'organization', 'indiana', 'university', 'lines', 'article', 'writes', 'paul', 'conditt', 'writes', 'insert', 'deletion', 'paul', 'aaron', 'discourse', 'anger', 'ref', 'galatians', 'know', 'obvious', 'not', 'speaking', 'acts', 'flesh', 'speaking', 'emotions', 'emotions', 'not', 'moral', 'immoral', 'good', 'bad', 'emotions', 'first', 'step', 'not', 'label', 'emotion', 'good', 'bad', 'numb', 'hide', 'true', 'feelings', 'accept', 'god', 'accepts', 'us', 'oh', 'definitely', 'please', 'look', 'colossians', 'ephesians', 'emotions', 'controlled', 'god', 'puts', 'strong', 'emphasis', 'otherwise', 'would', 'paul', 'write', 'timothy', 'much', 'making', 'sure', 'teach', 'insert', 'deletion', 'remainder', 'paragraph', 'aaron', 'quick', 'judge', 'forgiven', 'aids', 'dealt', 'taken', 'responsibility', 'feelings', 'made', 'appropriate', 'choices', 'action', 'feelings', 'not', 'given', 'anger', 'please', 'joe', 'issue', 'especially', 'feelings', 'action

In [ ]:
print(getTags(dataset))

['IN', ':', 'NN', 'VBZ', 'NN', '(', 'JJ', 'NN', 'NN', ')', 'NN', ':', 'NN', ':', 'NN', 'NN', ':', 'NN', 'NN', 'NNS', ':', 'CD', 'IN', 'NN', 'NNP', 'NN', 'NNP', 'NN', 'NNP', 'NN', 'NNP', 'NN', 'VBZ', ':', 'NN', 'NN', 'NN', 'NN', 'VBZ', ':', 'NN', 'JJ', 'NN', 'IN', 'NN', 'POS', 'CC', 'NN', 'POS', 'NN', 'IN', 'NN', ',', 'JJ', 'NNS', 'JJ', 'NNP', 'NNP', 'NNP', 'NN', 'VBP', 'RB', 'VB', 'WRB', 'PRP', 'VBZ', 'RB', 'JJ', '.', 'PRP', 'VBP', 'RB', 'VBG', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'NN', '.', 'PRP', 'VBP', 'RB', 'VBG', 'IN', 'NNS', '.', 'NNS', 'VBP', 'RB', 'IN', 'PRP', 'JJ', 'JJ', 'CC', 'JJ', ',', 'JJ', 'CC', 'JJ', '.', 'NNS', 'RB', 'VBP', '.', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'RB', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'JJ', 'CC', 'JJ', 'CC', 'TO', 'VB', 'PRP', 'RB', 'IN', 'NN', 'PRP', 'VBP', 'PRP$', 'JJ', 'NNS', ',', 'PRP', 'VBZ', 'TO', 'VB', 'PRP', 'IN', 'PRP', 'VBP', ',', 'IN', 'JJ', 'NN', 'NNS', 'PRP', '.', 'UH', ',', 'CC', 'PRP', 'RB', 'MD', 'VB', '.', 'VB', 'NN', 'IN', 'NNS', 'JJ', 'CC', 'VBZ

# Step 2: Build a Pipeline

**Modify the block code below to your choice of classifier [link text](https://https://www.nltk.org/book/ch06.html) **

In [ ]:
print(categories1)


['sci.med', 'alt.atheism']


In [ ]:
#twenty_train1 = fetch_20newsgroups(subset='train',  categories=categories1, shuffle=True, random_state=42)
#twenty_test1 = fetch_20newsgroups(subset='test',  categories=categories1, shuffle=True, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression

# Level: lexicon, model: tf-idf
text_clf = Pipeline([
    # add your code about text processing           
    ('vect', CountVectorizer(analyzer=processText)), 
    ('tfidf', TfidfTransformer(use_idf=True)),

    # change your classifier here, search: sklearn logistic regression example
    ('clf', SGDClassifier())
   #  ('clf', LogisticRegression())

])

In [ ]:
# To train the model 
text_clf.fit(twenty_train.data, twenty_train.target)
 

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function processText at 0x7f449db58560>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None

# Step 3: Make Prediction

In [ ]:
# To make prediction with dev/test set
predicted = text_clf.predict(twenty_test.data)

# Step 4: Evaluation

In [ ]:
# To evaluate your prediction on dev set
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(twenty_test.target, predicted))

print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

# confusion class
pd.DataFrame(metrics.confusion_matrix(twenty_test.target, predicted),
             columns=twenty_test.target_names,index=twenty_test.target_names)

Accuracy: 0.9134487350199734
                        precision    recall  f1-score   support

           alt.atheism       0.90      0.82      0.86       319
         comp.graphics       0.93      0.96      0.94       389
               sci.med       0.95      0.92      0.93       396
soc.religion.christian       0.88      0.94      0.91       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



,alt.atheism,comp.graphics,sci.med,soc.religion.christian
alt.atheism,261,4,10,44
comp.graphics,8,373,6,2
sci.med,9,17,363,7
soc.religion.christian,11,7,5,375


# Step 5: Error Analysis and Discussion
write down your own obseration about the predictions. Consider both confusion matrix and selected examples. Which classes are predicted correctly or incorrecly, possible explaination, possible solutions 

Exmaple: 1) Lab Practical, which feature is helpful for female name classification. https://www.nltk.org/book/ch06.html 
2) research paper: https://github.com/yoonkim/CNN_sentence 


In [ ]:
df_pred = pd.DataFrame({'news':twenty_test.data,'prediction':predicted, 'true':twenty_test.target})
df_pred[df_pred['true'] != df_pred['prediction']]

,news,prediction,true
12,"From: ""Gabriel D. Underwood"" <gabe+@CMU.EDU>\n...",3,2
15,From: swf@elsegundoca.ncr.com (Stan Friesen)\n...,2,0
19,From: mathew <mathew@mantis.co.uk>\nSubject: D...,3,0
26,From: UC512052@mizzou1.missouri.edu (David K. ...,2,1
36,Organization: Penn State University\nFrom: <RF...,0,2
...,...,...,...
1438,From: mangoe@cs.umd.edu (Charley Wingate)\nSub...,0,3
1450,From: alan.barclay@almac.co.uk (Alan Barclay)\...,1,2
1455,From: pww@spacsun.rice.edu (Peter Walker)\nSub...,3,0
1480,From: wilsonr@logica.co.uk\nSubject: Re: What ...,3,0


#References:  

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html 

https://www.nltk.org/book/ch06.html 

search: Other online resources: 

https://towardsdatascience.com/setting-up-text-preprocessing-pipeline-using-scikit-learn-and-spacy-e09b9b76758f 
 
sentiment analysis scikit learn 

scikit learn or nltk + NLP techniques 

python + NLP techniques

scikit learn logistic regression


